Implementing a CartesI/O controller
======

In this notebook we learn how to implement a Cartesian controller inside the *CartesI/O* framework. As an IK engine, we will use the *OpenSoT*-based `OpenSoT::IkProblem` class of the `tutorial_iros2018/opensot` notebook.

The advantage of doing so is the whole set of **auto-generated ROS tools** that can be used to customize the solver behavior from a remote process.

## Creating the class

First, we create an header file (e.g. `SimpleIkController.h`). Inside such a file, we create a new class (e.g. `SimpleIk`) that **derives from XBot::Cartesian::CartesianInterfaceImpl**.

We need to **implement an `update()` method**, that actually solves the IK problem according to our formulation of choice. Its signature is 
```
virtual bool update(double time, double period) override;
```
as it can be checked from the `cartesian_interface/CartesianInterfaceImpl.h` header file.

Optionally, a `setBaseLink(...)` can be overridden in order to enable run-time customization of the base-link of each defined task.

The base class *XBot::Cartesian::CartesianInterfaceImpl* is constructed using a *XBot::ModelInterface* object (that stores the IK solution state and provides the robot kinematics/dynamics) and a *ProblemDescription* object which describes the IK problem to be solved.

**NOTE:** Because our `OpenSoT::IkProblem` class solves an hard-coded IK problem, we only require the model in order to construct our `SimpleIk` class.

## Implementation
### Constructor
The base class constructor is called. Provided arguments are
 - the model 
 - an hard-coded problem description object
 
Finally, the `OpenSoT::IkProblem` class is created.

### Update method
- First, the **base class update** is called. 
- Then, current references are queried for each defined task (`getPoseReference()`), including the postural task (`getReferencePosture`). **NOTE:** the resulting references have been **automatically interpolated** by the base class, such that hard velocity/acceleration limits are satisfied.
- The IK problem is updated and solved.
- The solution is stored inside the model object.

Check the **provided reference implementation** `SimpleIkController.cpp` for details! In particolar, note that some code is required to make the solver "discoverable" by the CartesI/O framework. More specifically, we require methods to construct and destroy the solver, as follows:
```
extern "C" XBot::Cartesian::CartesianInterface* create_instance(XBot::ModelInterface::Ptr model,
                                                                XBot::Cartesian::ProblemDescription pb)
{
    return new XBot::Cartesian::SimpleIk(model);
}

extern "C" void destroy_instance( XBot::Cartesian::CartesianInterface* instance )
{
    delete instance;
}

```